Shows how one can generate text given a prompt and some hyperparameters, using either minGPT or huggingface/transformers

In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

ModuleNotFoundError: No module named 'torch'

In [ ]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2-xl'
device = 'cuda'

: 

In [ ]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

number of parameters: 1557.61M


In [ ]:

def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)
        

In [ ]:
generate(prompt='Andrej Karpathy, the', num_samples=10, steps=20)

--------------------------------------------------------------------------------
Andrej Karpathy, the chief of the criminal investigation department, said during a news conference, "We still have a lot of
--------------------------------------------------------------------------------
Andrej Karpathy, the man whom most of America believes is the architect of the current financial crisis. He runs the National Council
--------------------------------------------------------------------------------
Andrej Karpathy, the head of the Department for Regional Reform of Bulgaria and an MP in the centre-right GERB party
--------------------------------------------------------------------------------
Andrej Karpathy, the former head of the World Bank's IMF department, who worked closely with the IMF. The IMF had
--------------------------------------------------------------------------------
Andrej Karpathy, the vice president for innovation and research at Citi who oversaw the team's work to mak